# Projet DALAS
## Données entre 2014-2024 (exclu)
### SOYKOK Aylin 28711545 - CELIK Simay 28713301


<b>
Cette notebook contient le code afin de scraper des données de "quality of life" pour plusieurs pays entre plusieurs années depuis le site numbeo. Le document est en franglais, il faut le corriger.

TO DO:
data cleaning

In [2]:
import bs4
import lxml
import pandas as pd
import urllib

from urllib import request

In [3]:
years = [i for i in range(2015,2025)]
link = "https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2014"
req = request.urlopen(link).read()
page = bs4.BeautifulSoup(req, "lxml")
table = page.find('table',{"id":"t2"})
headers = [th.text.rstrip() for th in table.find_all('th')]
headers.append("Year")
rows = table.find_all('tr')
list_data = []
for row in rows[1:]:
  data = [d.text.rstrip() for d in row.find_all('td')]
  data.append(2014)
  list_data.append(data)

for year in years:
  link = "https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title="+str(year)
  req = request.urlopen(link).read()
  page = bs4.BeautifulSoup(req, "lxml")
  table = page.find('table',{"id":"t2"})
  rows = table.find_all('tr')
  for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    data.append(year)
    list_data.append(data)

In [9]:
df = pd.DataFrame(list_data,columns=headers)
#df=df.sort_values(by='Year')
#df.set_index('Rank',inplace=True)

df.columns
df.drop(columns='Rank',inplace=True)

In [10]:
df.head()

,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Year
0,Switzerland,206.2,138.1,68.2,68.0,143.9,7.1,24.6,26.8,-,2014
1,United States,195.5,132.9,49.9,68.6,77.4,2.4,37.1,35.1,-,2014
2,Germany,192.7,112.3,72.9,75.3,87.1,5.6,36.1,30.2,-,2014
3,Sweden,180.9,106.1,61.7,75.2,103.7,9.4,34.1,17.5,-,2014
4,Finland,178.9,97.9,70.8,68.7,103.3,7.8,37.0,16.7,-,2014


In [11]:
df_nan =  df.isna()
nan_lines = df_nan.any(axis=1)
nan_lines[nan_lines==True]

Series([], dtype: bool)

Aucun NaN.

For missing values (some countries do not exist in some years) we can detect which country is missing in which year and then for all the columns we can put the mean of the previous and the next year. Using adjacent years would prevent weird points in a graph. We can also mark these "estimations" to know which were generated by us and which were in the original source. => Ask the prof.

<b> Cleaning missing values


In [12]:
df['Generated'] = 0 #1 si généré par nous, 0 sinon

def fill_missing(group):
    """Trouve les lignes manquantes, les remplit avec la moyenne des années adjacentes."""
    missing = set(df['Country'].unique()) - set(group['Country'].unique()) #pays manquants par groupe d'année
    for m in missing:
        prev = df[(df['Country']==m) & (df['Year']==group['Year'].min() - 1)]
        next = df[(df['Country']==m) & (df['Year']==group['Year'].max() + 1)]
        if not prev.empty and not prev.empty:
            moy = (prev.iloc[:, 1:-1] + prev.iloc[:, 1:-1]) / 2
            moy['Year'] = group['Year']
            moy['Country'] = m
            moy['Generated'] = 1 #1 si généré par nous
            group = pd.concat([group, moy], ignore_index=True)
    return group
        

filled_df = df.groupby('Year').apply(fill_missing).reset_index(drop=True)
filled_df

In [13]:
df.to_csv('../data/data_annees.csv', index=False)